**计算给定数据集的信息熵** \
$$ H = -\sum^n_{i=1}p(x_i)log_2p(x_i)     \tag{香农熵} $$

In [7]:
import operator
from math import log
import numpy as np

def calcShannoEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLable = featVec[-1]
        if currentLable not in labelCounts.keys():
            labelCounts[currentLable] = 0
        labelCounts[currentLable] += 1
    shannoEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannoEnt -= prob * log(prob, 2)
    return shannoEnt

以上代码是将每个类别出现的次数，以k-v的方式保存在字典，并按照公式计算出熵值

In [26]:
# Example
def createDateSet():
    dataSet = [[1,1,'y'],[1,1,'y'],[1,0,'n'],[0,1,'n'],[0,1,'n']]
    labels = ['no surfaceing', 'flippers']
    return dataSet, labels

mydata,mylabels=createDateSet()
mydata,mylabels

([[1, 1, 'y'], [1, 1, 'y'], [1, 0, 'n'], [0, 1, 'n'], [0, 1, 'n']],
 ['no surfaceing', 'flippers'])

In [3]:
calcShannoEnt(mydata)

0.9709505944546686

In [4]:
mydata[0][-1]='maybe'
mydata,calcShannoEnt(mydata)

([[1, 1, 'maybe'], [1, 1, 'y'], [1, 0, 'n'], [0, 1, 'n'], [0, 1, 'n']],
 1.3709505944546687)

可以看到 在增加更多的分类后，熵值增加了，也就是数据的混乱程度增加了

得到熵值以后，我们就可以按照最大信息增益来划分数据集了，再此之前补充两个知识，extend 与 append的区别

In [5]:
a = [1,2,3];b=[4,5,6]
a.append(b)
a

[1, 2, 3, [4, 5, 6]]

In [6]:
a=[1,2,3]
a.extend(b)
a

[1, 2, 3, 4, 5, 6]

当我们按照某个特征划分数据集的时候，就是将所有的，符合条件的元素都抽取出来

In [181]:
def splitDataSet(dataSet, axis, value):
    retDataSet1 = []
    retDataSet2 = []
    for featVec in dataSet:
        if float(featVec[axis]) >= value:
            retDataSet1.append(featVec)
        if float(featVec[axis]) < value:
            retDataSet2.append(featVec)
    return retDataSet1,retDataSet2

In [8]:
# Example
data1=splitDataSet(mydata,0,1)
data2=splitDataSet(mydata,0,0)
data1,data2

([[1, 'maybe'], [1, 'y'], [0, 'n']], [[1, 'n'], [1, 'n']])

现在已经找到按指定特征划分数据集的方法了，下一步就是寻找一种最好的划分数据集发的方式\
最好的数据集划分方式要使得划分后的信息增益最大（ID3）

In [9]:
def chooseBestFeatSplit(dataSet):
    numFeatures = len(dataSet[0])-1 # -1的原因是减去一列的分类，获取特征的类数
    baseEnt = calcShannoEnt(dataSet) # 计算最开始的熵值
    bestInfoGain = 0.0
    bestFeat = -1
    for i in range(numFeatures):
        featList = [a[i] for a in dataSet] # 把第i类特征的所有可能取值放入featList
        uniqueVals = set(featList) # set集合去重
        newEnt = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value) # 划分数据集
            prob = len(subDataSet)/float(len(dataSet)) # 计算概率
            newEnt += prob * calcShannoEnt(subDataSet) # 计算新熵值
        infoGain = baseEnt - newEnt # 计算信息增益
        if(infoGain > bestInfoGain): #记录最佳信息增益的特征
            bestInfoGain = infoGain
            bestFeat = i
    return bestFeat

In [10]:
chooseBestFeatSplit(mydata) # Test

0

**递归构建决策树**

递归结束的条件
- 遍历完所有数据集属性
- 每个分支下的所有实例都具有相同的分类

但是在一些情况下，用完所以的属性，但类标签仍然不是唯一，这个时候采用多数表决方式决定该节点的分类

In [10]:
def majorityCnt(classList): #类似于knn决策函数
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

**现在可以开始构建决策树了**

In [11]:
def createTree(dataSet,labels):
    classList = [a[-1] for a in dataSet] # 获取所有类标签
    if classList.count(classList[0]) == len(classList): 
        return classList[0]  # 如果类别完全相同，停止继续划分
    if len(dataSet[0]) == 1:
        return majorityCnt(classList) # 如果遍历完所有特征，则返回出现类别最多的类别
    bestFeatIndex = chooseBestFeatSplit(dataSet) # 选择最好的特征进行划分
    bestFeatLabel = labels[bestFeatIndex]
    mytree = {bestFeatLabel:{}}
    del(labels[bestFeatIndex]) # 删除已使用特征
    featValues = [a[bestFeatIndex] for a in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:] # 参数按引用方式传递，直接传会改变lables的值，故复制一份
        mytree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeatIndex,value),subLabels) # 递归调用
    return mytree

In [28]:
# Test

mydata,mylabels=createDateSet()
myTree= createTree(mydata,mylabels)
myTree

{'no surfaceing': {0: 'n', 1: {'flippers': {0: 'n', 1: 'y'}}}}

In [242]:

# 使用决策树执行分类
def classify(inputTree,featLabels,testVec):
    carLabel = 'error'
    firstStr = list(inputTree.keys())[0] # 找到根节点
    secondDict = inputTree[firstStr] # 获取子树
    featIndex = featLabels.index(firstStr) # 找到划分数据集的特征在实际数据集中的位置（索引方式）
    for key in secondDict.keys():
        if float(testVec[featIndex]) >= float(key[1:]) and key[0] == '>' and type(secondDict[key]) == dict: 
             carLabel = classify(secondDict[key],featLabels,testVec)
        elif float(testVec[featIndex]) < float(key[1:]) and key[0] == '<' and type(secondDict[key]) == dict: 
             carLabel = classify(secondDict[key],featLabels,testVec)
        elif float(testVec[featIndex]) >= float(key[1:]) and key[0] == '>' and type(secondDict[key]) != dict: 
            carLabel=secondDict[key]
        elif float(testVec[featIndex]) < float(key[1:]) and key[0] == '<' and type(secondDict[key]) != dict: 
            carLabel=secondDict[key]         
    return carLabel

In [30]:
mydata,mylabels=createDateSet()
classify(myTree,mylabels,[1,0])

'n'

In [7]:
#读入数据
fr=open('lenses.txt')
lenses=[i.strip().split('\t') for i in fr.readlines()]
lenses

[['young', 'myope', 'no', 'reduced', 'no lenses'],
 ['young', 'myope', 'no', 'normal', 'soft'],
 ['young', 'myope', 'yes', 'reduced', 'no lenses'],
 ['young', 'myope', 'yes', 'normal', 'hard'],
 ['young', 'hyper', 'no', 'reduced', 'no lenses'],
 ['young', 'hyper', 'no', 'normal', 'soft'],
 ['young', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['young', 'hyper', 'yes', 'normal', 'hard'],
 ['pre', 'myope', 'no', 'reduced', 'no lenses'],
 ['pre', 'myope', 'no', 'normal', 'soft'],
 ['pre', 'myope', 'yes', 'reduced', 'no lenses'],
 ['pre', 'myope', 'yes', 'normal', 'hard'],
 ['pre', 'hyper', 'no', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'no', 'normal', 'soft'],
 ['pre', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'yes', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'normal', 'hard'],
 ['presbyopic', 

In [27]:
lenseLables=['age','prescript','astigmatic','tearRate']
lenseTree =createTree(lenses,lenseLables)
lenseTree

{'tearRate': {'normal': {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'hyper': 'soft',
        'myope': 'no lenses'}},
      'young': 'soft',
      'pre': 'soft'}},
    'yes': {'prescript': {'hyper': {'age': {'presbyopic': 'no lenses',
        'young': 'hard',
        'pre': 'no lenses'}},
      'myope': 'hard'}}}},
  'reduced': 'no lenses'}}

In [8]:
def createTree_Fix(dataSet,labels):
    classList = [a[-1] for a in dataSet] # 获取所有类标签
    if len(dataSet)<=3 : return majorityCnt(classList)# 某样本数小于等于3不再分裂 直接多数表决
    if classList.count(classList[0]) == len(classList): 
        return classList[0]  # 如果类别完全相同，停止继续划分
    if len(dataSet[0]) == 1:
        return majorityCnt(classList) # 如果遍历完所有特征，则返回出现类别最多的类别
    bestFeatIndex = chooseBestFeatSplit(dataSet) # 选择最好的特征进行划分
    bestFeatLabel = labels[bestFeatIndex]
    mytree = {bestFeatLabel:{}}
    del(labels[bestFeatIndex]) # 删除已使用特征
    featValues = [a[bestFeatIndex] for a in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:] # 参数按引用方式传递，直接传会改变lables的值，故复制一份
        mytree[bestFeatLabel][value]=createTree_Fix(splitDataSet(dataSet,bestFeatIndex,value),subLabels) # 递归调用
    return mytree

In [9]:
lenseLables=['age','prescript','astigmatic','tearRate']
lenseTree_fix =createTree_Fix(lenses,lenseLables)
lenseTree_fix

{'tearRate': {'reduced': 'no lenses',
  'normal': {'astigmatic': {'yes': {'prescript': {'hyper': 'no lenses',
      'myope': 'hard'}},
    'no': {'age': {'presbyopic': 'no lenses',
      'young': 'soft',
      'pre': 'soft'}}}}}}

In [341]:
#读入数据
mylist=[]
fr=open('iris.data')
irisSet=[i.strip().split(',') for i in fr.readlines()]
np.random.shuffle(irisSet)

In [188]:
def chooseBestFeatSplit1(dataSet):
    numFeatures = len(dataSet[0])-1 # -1的原因是减去一列的分类，获取特征的类数
    baseEnt = calcShannoEnt(dataSet) # 计算最开始的熵值
    bestInfoGain = 0.0
    bestFeat = -1
    bestpoint = 0
    splitpoint=0
    for i in range(numFeatures):
        featList = [float(a[i]) for a in dataSet] # 把第i类特征的所有可能取值放入featList
        uniqueVals = set(featList)# set集合去重
        uniqueVals = list(uniqueVals)
        uniqueVals=sorted(uniqueVals)
        bestpoint,InfoGain= chooseBestpoint(i,uniqueVals,dataSet)
        if(InfoGain > bestInfoGain): #记录最佳信息增益的特征
            bestInfoGain = InfoGain
            bestFeat = i
            splitpoint=bestpoint
    return bestFeat,splitpoint
    

In [187]:
def chooseBestpoint(axis,uniqueVals,dataSet):
    baseEnt = calcShannoEnt(dataSet)# 计算最开始的熵值
    bestInfoGain = 0.0
    bestpoint = -1
    uniqueVals=list(uniqueVals)
    newsplitpoint=[(float(uniqueVals[i])+float(uniqueVals[i+1]))/2.0 for i in range(len(uniqueVals)-1)]
    for value in newsplitpoint:
            subDataSet1,subDataSet2 = splitDataSet(dataSet,axis,value) # 划分数据集
            prob1 = len(subDataSet1)/float(len(dataSet)) # 计算概率
            prob2 = len(subDataSet2)/float(len(dataSet)) 
            newEnt = prob1 * calcShannoEnt(subDataSet1)+prob2*calcShannoEnt(subDataSet2) # 计算新熵值
            infoGain = baseEnt - newEnt # 计算信息增益
            if(infoGain > bestInfoGain): #记录最佳信息增益的特征
                bestInfoGain = infoGain
                bestpoint = value 
    return bestpoint,bestInfoGain
    

In [219]:
def createTree1(dataSet,labels):
    classList = [a[-1] for a in dataSet] # 获取所有类标签
    if classList.count(classList[0]) == len(classList): 
        return classList[0]  # 如果类别完全相同，停止继续划分
    if len(dataSet[0]) == 1:
        return majorityCnt(classList) # 如果遍历完所有特征，则返回出现类别最多的类别
    bestFeatIndex,splitpoint = chooseBestFeatSplit1(dataSet) # 选择最好的特征进行划分
    bestFeatLabel = labels[bestFeatIndex]
    mytree = {bestFeatLabel:{}}
    sp1,sp2=splitDataSet(dataSet,bestFeatIndex,float(splitpoint))
    mytree[bestFeatLabel]['>{}'.format(splitpoint)]=createTree1(sp1,labels)
    mytree[bestFeatLabel]['<{}'.format(splitpoint)]=createTree1(sp2,labels)
                                                                       # 递归调用
    return mytree
 

In [336]:
def createTree2(dataSet,labels):
    classList = [a[-1] for a in dataSet] # 获取所有类标签
    if len(dataSet)<=4 : return majorityCnt(classList)
    if classList.count(classList[0]) == len(classList): 
        return classList[0]  # 如果类别完全相同，停止继续划分
    if len(dataSet[0]) == 1:
        return majorityCnt(classList) # 如果遍历完所有特征，则返回出现类别最多的类别
    bestFeatIndex,splitpoint = chooseBestFeatSplit1(dataSet) # 选择最好的特征进行划分
    bestFeatLabel = labels[bestFeatIndex]
    mytree = {bestFeatLabel:{}}
    sp1,sp2=splitDataSet(dataSet,bestFeatIndex,float(splitpoint))
    mytree[bestFeatLabel]['>{}'.format(splitpoint)]=createTree2(sp1,labels)
    mytree[bestFeatLabel]['<{}'.format(splitpoint)]=createTree2(sp2,labels)
                                                                       # 递归调用
    return mytree

In [338]:

irislabels=['sepal length','sepal width','petal length','petal width','class']
trainSet=irisSet[50:]
iristree=createTree1(trainSet,irislabels)
isistree2=createTree2(trainSet,irislabels)



In [329]:
testSet=irisSet[:50]

In [308]:
def fun1():
    errorcnt=0.0
    for i in testSet:
        if classify(iristree,irislabels,i[:4]) != i[-1]:errorcnt += 1
    print(errorcnt)
    print(errorcnt/len(testSet))

In [309]:
def fun2():
    errorcnt=0.0
    for i in testSet:
        if classify(isistree2,irislabels,i[:4]) != i[-1]:errorcnt += 1
    print(errorcnt)
    print(errorcnt/len(testSet))

In [310]:
fun1(),fun2() #90train 60test

6.0
0.1
3.0
0.05


(None, None)

In [344]:
fun1(),fun2() #100train 50test

2.0
0.04
3.0
0.06


(None, None)

In [342]:
fun1(),fun2() #120train 30test

2.0
0.04
3.0
0.06


(None, None)

In [353]:
#读入数据
mylist=[]
fr=open('iris.data')
irisSet=[i.strip().split(',') for i in fr.readlines()]
np.random.shuffle(irisSet)
irislabels=['sepal length','sepal width','petal length','petal width','class']
trainSet=irisSet[60:]
iristree=createTree1(trainSet,irislabels)
iristree
isistree2=createTree2(trainSet,irislabels)
testSet=irisSet[:60]
fun1(),fun2() #100train 50test

4.0
0.06666666666666667
2.0
0.03333333333333333


(None, None)

In [354]:
iristree

{'petal length': {'>2.45': {'petal width': {'>1.7000000000000002': {'petal length': {'>4.85': 'Iris-virginica',
      '<4.85': {'sepal length': {'>5.95': 'Iris-virginica',
        '<5.95': 'Iris-versicolor'}}}},
    '<1.7000000000000002': {'petal length': {'>4.95': {'sepal length': {'>6.6': 'Iris-virginica',
        '<6.6': {'sepal width': {'>2.45': 'Iris-versicolor',
          '<2.45': 'Iris-virginica'}}}},
      '<4.95': 'Iris-versicolor'}}}},
  '<2.45': 'Iris-setosa'}}

In [355]:
isistree2

{'petal length': {'>2.45': {'petal width': {'>1.7000000000000002': {'petal length': {'>4.85': 'Iris-virginica',
      '<4.85': 'Iris-virginica'}},
    '<1.7000000000000002': {'petal length': {'>4.95': 'Iris-virginica',
      '<4.95': 'Iris-versicolor'}}}},
  '<2.45': 'Iris-setosa'}}